# The Wat
For this project we will be using the jokes on /r/jokes subreddit as our dataset. 

Only "funny" jokes will be saved, since the last thing we want from our model is to make lame jokes... although I just realized that a dad jokes bot would be pretty hilarious too :D. Here, we'll select all the joke submissions with a number of upvotes (score) higher than a threshold of 420 (nice).


There are 2 (afaik) to retrieve this data:

- Using the praw lib (Python Reddit API Wrapper).
- Pushshift Reddit API, which is a RESTful API.

## PRAW

This is the easiest way of accessing reddit content with python. However, ever since reddit introduced a limit of use for its API, only 100 submissions can be retrieved per query.

In [74]:
import praw
import pandas as pd
from tqdm import tqdm

reddit = praw.Reddit(client_id = 'b1pL2McByh6fAQ',
                    client_secret = '409PPLrvLmZp14C6Q48RVr_3Bmr0bA',
                    username = 'yourejokeguy',
                    password = 'yo-5mmL8#~H}',
                    user_agent = 'joker')

rjokes = reddit.subreddit('jokes')

Version 5.4.0 of praw is outdated. Version 7.1.0 was released Tuesday June 23, 2020.


In [2]:
top = rjokes.top(limit=100000)


jokeset = pd.DataFrame(columns=['Title', 'Text'])

pbar = tqdm(top, desc='Getting the jokes...')

for submission in pbar:
    jokeset = jokeset.append({'Title': submission.title, 'Text': submission.selftext}, ignore_index=True)

Getting the jokes...: 998it [00:26, 37.21it/s]


In [3]:
jokeset.head()

,Title,Text
0,Sad News: The founder of /r/jokes has passed away,"RIP Larry Tesler, the UI designer that created..."
1,Calm down about the Net Neutrality thing...,Paying additional money to access certain site...
2,V,V\n\n*Edit: seems like the ctrl key on my keyb...
3,If your surprised that Jeffrey Epstein commite...,Imagine how surprised he must have been.\n\nEd...
4,A new Navy recruit has his first day on the su...,"He speaks with the officer, who assigns him hi..."


In [4]:
jokeset = pd.DataFrame(columns=['Title', 'Text'])

p_bar = tqdm(psraw.submission_search(reddit, subreddit='jokes', limit=10000))
for item in p_bar:
    jokeset = jokeset.append({'Title': item.title, 'Text': item.selftext}, ignore_index=True)

100it [00:09, 10.16it/s]


In [6]:
jokeset

,Title,Text
0,A Russian goes into a Moscow department store ...,"The sales assistant says ""Sorry, there are no ..."
1,What does a bored trampoline jumper say?,"""Okay, bye. I'm gonna bounce now."""
2,My wife had been acting suspicious so I checke...,I think she had enough air.
3,What is a mathematician's favourite food?,What's a mathematician's favourite food? \n\nI...
4,What do you call six gay people in war?,Rainbow Six Siege.
...,...,...
95,Hot hand,[rimosso]
96,I think Bill Gates named his company after me...,... because my dick is micro and soft.
97,"A man calls up his lawyer and says, “How much ...",[removed]
98,On the outside I laugh laugh,[removed]


## Pushshift RESTful API

With this approach we can overcome the rather stringent limitation of the previous method.

Code adapted from: https://www.reddit.com/r/pushshift/comments/i8dlzs/how_to_download_all_posts_from_a_subreddit/

In [80]:
import requests, json
import time

subreddit = 'jokes'
min_score = 420
maxThings = -1
printWait = 2
requestSize = 100


def requestJSON(url):
    while True:
        try:
            r = requests.get(url)
            if r.status_code != 200:
                print('error code', r.status_code)
                time.sleep(5)
                continue
            else:
                break
        except Exception as e:
            print(e)
            time.sleep(5)
            continue
    return json.loads(r.text)

meta = requestJSON('https://api.pushshift.io/meta')
limitPerMinute = meta['server_ratelimit_per_minute']
requestWait = 60 / limitPerMinute

print('server_ratelimit_per_minute', limitPerMinute)

things = ['submission']

num = 0
for thing in things:
    i = 0
    
    df = pd.DataFrame(columns=['title', 'text'])
    print('\n[starting', thing + 's]')

    if maxThings < 0:

        url = 'https://api.pushshift.io/reddit/search/'\
              + thing + '/?subreddit=' + subreddit\
              + '&score=>' + str(min_score) \
              + '&metadata=true&size=0'

        meta_json = requestJSON(url)

        totalResults = meta_json['metadata']['total_results']
        print('total ' + thing + 's', 'in', subreddit,':', totalResults)
    else:
        totalResults = maxThings
        print('downloading most recent', maxThings)


    created_utc = ''

    startTime = time.time()
    timePrint = startTime
    while True:
        url = 'http://api.pushshift.io/reddit/search/'\
              + thing \
              + '?subreddit=' + subreddit \
              + '&score=>' + str(min_score) \
              + '&size=' + str(requestSize)\
              + '&before=' + str(created_utc)

        res_json = requestJSON(url)

        if len(res_json['data']) == 0:
            break

        doneHere = False
        for post in res_json['data']:
            created_utc = post["created_utc"]
            
            if 'selftext' not in post.keys():
                print('\n #No text#')
                print(post)
                continue
            joke = {'title': str(post['title']), 'text': str(post['selftext'])}
            df = df.append(joke, ignore_index=True)
            i += 1
            if i >= totalResults:
                doneHere = True
                break

        if doneHere:
            break

        if time.time() - timePrint > printWait:
            timePrint = time.time()
            percent = i / totalResults * 100

            timePassed = time.time() - startTime

            print('{:.2f}'.format(percent) + '%', '|',
                    time.strftime("%H:%M:%S", time.gmtime(timePassed)))


        time.sleep(requestWait)
    df.to_csv(f'data/{subreddit}_set.csv', index=False)
    num += 1

server_ratelimit_per_minute 120

[starting submissions]
total submissions in jokes : 23809
0.42% | 00:00:06
error code 522
0.84% | 00:00:43

 #No text#
{'all_awardings': [], 'allow_live_comments': True, 'author': '[deleted]', 'author_flair_background_color': '', 'author_flair_css_class': None, 'author_flair_text': None, 'author_flair_text_color': 'dark', 'awarders': [], 'banned_by': 'moderators', 'can_mod_post': False, 'contest_mode': False, 'created_utc': 1586778946, 'domain': 'self.Jokes', 'full_link': 'https://www.reddit.com/r/Jokes/comments/g0hjsf/the_best_joke_to_tell_at_parties/', 'gildings': {}, 'id': 'g0hjsf', 'is_crosspostable': False, 'is_meta': False, 'is_original_content': False, 'is_reddit_media_domain': False, 'is_robot_indexable': False, 'is_self': True, 'is_video': False, 'link_flair_background_color': '', 'link_flair_css_class': 'long', 'link_flair_richtext': [], 'link_flair_text': 'Long', 'link_flair_text_color': 'dark', 'link_flair_type': 'text', 'locked': False, 'me

In [82]:
df

,title,text
0,A 55 year old lady suddenly started learning t...,"Everyone was curious and asked her: ""why the ..."
1,A Roman Catholic priest is on his way to Rome ...,"“My goodness, Mary!” He says. “How have you be..."
2,How does an alchemist please his wife?,Elixer
3,Do you remember when air was free at the gas s...,Inflation
4,two hardcore trump supporters die and ascend t...,God meets them at the pearly gates and asks if...
...,...,...
23773,PRANK CALL I did when I was a kid.,"PHONE RINGS\n\nPerson Answers - ""Hello?""\n\nPr..."
23774,When Engineers go to hell...,An engineer dies and reports to the Pearly Gat...
23775,So I've been chatting with this 13 year old gi...,How cool is that for someone her age??
23776,And that's how the fight started.....\r\n,"HOW TO START A FIGHT\r\n\r\nOne year, I decide..."
